In [1]:
import pandas as pd
import numpy as np

In [2]:
def Convert(string):
    list1 = []
    list1[:0] = string.strip('\n')
    return list1

In [3]:
def one_d_visibility(row):
    # Create a series of the same size as the incoming row
    ser = pd.Series(True,range(row.shape[0]))
    
    # Loop through the row to find which is visible from one direction
    max_height = -1
    for idx, cell_value in row.items():
        cell_value = int(cell_value)
        if cell_value > max_height:
            max_height = cell_value
            ser[idx] = True
        else:
            ser[idx] = False
    return(ser)

In [4]:
'''
What's our Data?
'''

with open('data.txt') as infile:
    data = infile.readlines()

df_o = pd.DataFrame([Convert(x) for x in data])

'''
To get a 2d scan, need to scan the 1d visibility of 
the dataframe forward & backwards (for Left/Right)
then transpose (T) the dataframe to scan again
(for Up/Down)...and of course transpose it back at the end.
'''

vis_2d_parts = []
for df in [df_o, df_o.T]:

    tempdf = pd.DataFrame()
    for idx, row in df.iterrows():
        ser1 = one_d_visibility(row)
        row_r = row[::-1]
        ser2 = one_d_visibility(row_r)
        v = ser1 | ser2
        tempdf = pd.concat([tempdf, v],axis=1)    

    vis_2d_parts.append(tempdf)
    
'''
Re-index to make it pretty (so we can do our logical OR)
'''

vis_2d_parts[0] = vis_2d_parts[0].T.reset_index(drop=True)
vis_2d_parts[1] = vis_2d_parts[1].T.reset_index(drop=True).T  #See?  We transposed it back!

'''
Logical OR will show visibility from both up/down AND left/right
'''

vis = vis_2d_parts[0] | vis_2d_parts[1]

'''
Summing the dataframe will give us a series of the num visible in each ROW.
Summing AGAIN will give the total number of trees available.
'''

print("Total Visible Trees =", vis.sum().sum())

Total Visible Trees = 1690


In [5]:
def one_d_scenic_score(row):
    # Create a series of the same size as the incoming row
    ser = pd.Series(1,range(row.shape[0]))
    
    # Loop through the row to find which is visible from one direction    
    for idx, cell_value in row.items():
        if idx in (0, row.shape[0] - 1):
            ser[idx] = 0
        else:
            # LEFT slice
            left = row[:idx]
            left_seen = 0
            for l in left.iloc[::-1]:
                if l < cell_value:
                    left_seen += 1
                else:
                    left_seen += 1
                    break
            ser[idx] *= left_seen
            
            # RIGHT slice
            right = row[idx+1:]
            right_seen = 0
            for r in right.iloc[::]:
                if r < cell_value:
                    right_seen += 1
                else:
                    right_seen += 1
                    break
            ser[idx] *= right_seen            
    return(ser)

In [6]:
'''
What's our Data?
'''

with open('data.txt') as infile:
    data = infile.readlines()

df_o = pd.DataFrame([Convert(x) for x in data])

scenic_score_parts = []
for df in [df_o,df_o.T]:
    tempdf = pd.DataFrame()
    #print(df)
    
    for idx,row in df.iterrows():
        score = one_d_scenic_score(row)
        #print(score)
        tempdf = pd.concat([tempdf,score],axis=1)
        #print()
    #print()
    #print(tempdf)
    scenic_score_parts.append(tempdf)

'''
Re-index to make it pretty (so we can do our matrix multiplication)
Transpose things to get them in the right order...
'''
    
scenic_score_parts[0] = scenic_score_parts[0].T.reset_index(drop=True)
scenic_score_parts[1] = scenic_score_parts[1].T.reset_index(drop=True).T  

'''
Matrix Multiplication (NOT DOT PRODUCT!) to get the combined score matrix
'''

score = scenic_score_parts[0] * scenic_score_parts[1]

'''
Get the max of each column, then the max of those to get the best score!
'''

max_score = score.max().max()
print("Highest Scenic Score:", max_score)

Highest Scenic Score: 535680
